## San Francisco crime data analysis and travel tips recommondation

#### Using Spark SQL for big data analysis on SF crime data and travel tips recommondation
data are public available and can be download via (https://data.sfgov.org/Public-Safety/sf-data/skgt-fej3/data)

Author: Jiaxi Gao | gaojiaxi2015@gmail.com

- <a href='#1'>1. Introduction</a>
    - <a href='#1.1'>1.1. Target Overview</a>
    - <a href='#1.2'>1.2. Tools and Libraries</a>

- <a href='#2'>2. Exploratory SF Crime Data Analysis and Travel Safety Tips</a>
    - <a href='#2.1'>2.1. Data Overview</a>
    - <a href='#2.2'>2.2. Number of crime for each category</a>
    - <a href='#2.3'>2.3. Number of crime for each Sunday</a>
    - <a href='#2.4'>2.4. Number of crime in each month</a>
    - <a href='#2.5'>2.5. Number of crime on Christmas holiday</a>
    - <a href='#2.6'>2.6. Top-3 dangerous district and the corresponding most frequent types of crimes</a>
    - <a href='#2.7'>2.7. Resolution of Unsolved crime cases</a>
    - <a href='#2.8'>2.8. My conslusion for travel in SF</a>
    
- <a href='#3'>3. Venue Recommandation</a>
    - <a href='#3.1'>3.1. Hotel Booking Recommandation</a>
    - <a href='#3.2'>3.2. Venue Exploring and Recommandation using Foursquare API</a>

- <a href='#4'>4. Conclusion</a>

# <a id='1'>1. Introduction</a>

### <a id='1.1'>1.1. Target overview</a>

In this notebook we are going to analyse San Francisco crime data from 2003 to 2018. The aim of this notebook is providing residents in SF and tourists with travel safety tips and business insights based on modeling and analysis of most common types of crime, top dangerous neighbourhoods, and resolved/unresolved rates for different types of crimes etc. In addition, we aim to provide tourists with some venues they may visit during their trip in SF using the social location service platforms such as Yelp API or Foursquare API. The venues we recommand will located in safe neighborhood in SF and will have high positive reviews by other customers.

### <a id='1.2'>1.2. Tools and Libraries</a>

We will use <span style="color:red">Spark</span> and <span style="color:red">Spark SQL</span> to perform big data analysis as it is designed for analysis Massive data sets in a distributed fashion.
We will use <span style="color:red">Foursquare API</span> to perform venue recommandations. The doc for Foursquare API can be found using this [link](https://developer.foursquare.com/)

# <a id='2'>2. Exploratory SF Crime Data Analysis and Travel Safety Tips</a>

### <a id='2.1'>2.1. Data Overview</a>

In [1]:
from csv import reader
from pyspark import SparkContext
from pyspark.sql import Row 
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import to_date, to_timestamp, year, month, dayofmonth, hour, minute
from pyspark.sql.functions import udf, lit
import pyspark.sql.functions as fn
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import warnings
import math
from ggplot import *

import os


In [2]:
sc = SparkContext("local", "CrimeAnalysis")

In [3]:
#import urllib.request
#urllib.request.urlretrieve("https://data.sfgov.org/api/views/tmnf-yvry/rows.csv?accessType=DOWNLOAD", "./data/sf_03_18.csv")

In [4]:
data_path = "./data/sf_03_18.csv"

In [5]:
#Read data from the data storage
#Split the header by its separator
crime_data_lines = sc.textFile(data_path)
#prepare data: remove "
df_crimes = crime_data_lines.map(lambda line: [x.strip('"') for x in next(reader([line]))])
#get header
header = df_crimes.first()
print(header)

#remove the first line of data
crimes = df_crimes.filter(lambda x: x != header)

#get the total number of data 
print(" The crimes dataframe has {} records".format(crimes.count()))


['IncidntNum', 'Category', 'Descript', 'DayOfWeek', 'Date', 'Time', 'PdDistrict', 'Resolution', 'Address', 'X', 'Y', 'Location', 'PdId']
 The crimes dataframe has 2215024 records


In [6]:
from pyspark.sql import SparkSession
spark = SparkSession \
    .builder \
    .appName("CrimeAnalysis") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

#Load .csv
df_opt1 = spark.read.format("csv").option("header", "true").load(data_path)

#create temp table
df_opt1.createOrReplaceTempView("sf_crime")

#display first 10 rows
first_ten_rows = spark.sql("SELECT * FROM sf_crime LIMIT 10")
first_ten_rows.show()


+----------+--------------------+--------------------+---------+----------+-----+----------+--------------------+--------------------+-----------------+----------------+--------------------+--------------+
|IncidntNum|            Category|            Descript|DayOfWeek|      Date| Time|PdDistrict|          Resolution|             Address|                X|               Y|            Location|          PdId|
+----------+--------------------+--------------------+---------+----------+-----+----------+--------------------+--------------------+-----------------+----------------+--------------------+--------------+
| 110308742|      MISSING PERSON|        FOUND PERSON|   Friday|04/15/2011|16:00|   TARAVAL|             LOCATED|0 Block of CRESPI DR|-122.476327815126|37.7196874952717|POINT (-122.47632...|11030874275000|
| 090316914|            WARRANTS|      WARRANT ARREST| Thursday|03/26/2009|12:28|  SOUTHERN|      ARREST, BOOKED|1000 Block of MAR...|-122.411339562557|37.7812707434494|POINT (

In [7]:
#display(first_ten_rows)


### <a id='2.2'>2.2. Number of crime for each category</a>

In [8]:
s1_result = df_opt1.groupBy('category').count().orderBy('count', ascending=False)

#display result
s1_result.show()

+--------------------+------+
|            category| count|
+--------------------+------+
|       LARCENY/THEFT|480448|
|      OTHER OFFENSES|309358|
|        NON-CRIMINAL|238323|
|             ASSAULT|194694|
|       VEHICLE THEFT|126602|
|       DRUG/NARCOTIC|119628|
|           VANDALISM|116059|
|            WARRANTS|101379|
|            BURGLARY| 91543|
|      SUSPICIOUS OCC| 80444|
|      MISSING PERSON| 64961|
|             ROBBERY| 55867|
|               FRAUD| 41542|
|     SECONDARY CODES| 25831|
|FORGERY/COUNTERFE...| 23050|
|         WEAPON LAWS| 22234|
|            TRESPASS| 19449|
|        PROSTITUTION| 16701|
|     STOLEN PROPERTY| 11891|
|SEX OFFENSES, FOR...| 11742|
+--------------------+------+
only showing top 20 rows



In [9]:
#Spark SQL based
##in descending order
crimeCategory_desc = spark.sql('''
                SELECT category, COUNT(*) AS count 
                FROM sf_crime 
                GROUP BY category 
                ORDER BY count DESC''')

#display result
crimeCategory_desc.show()

+--------------------+------+
|            category| count|
+--------------------+------+
|       LARCENY/THEFT|480448|
|      OTHER OFFENSES|309358|
|        NON-CRIMINAL|238323|
|             ASSAULT|194694|
|       VEHICLE THEFT|126602|
|       DRUG/NARCOTIC|119628|
|           VANDALISM|116059|
|            WARRANTS|101379|
|            BURGLARY| 91543|
|      SUSPICIOUS OCC| 80444|
|      MISSING PERSON| 64961|
|             ROBBERY| 55867|
|               FRAUD| 41542|
|     SECONDARY CODES| 25831|
|FORGERY/COUNTERFE...| 23050|
|         WEAPON LAWS| 22234|
|            TRESPASS| 19449|
|        PROSTITUTION| 16701|
|     STOLEN PROPERTY| 11891|
|SEX OFFENSES, FOR...| 11742|
+--------------------+------+
only showing top 20 rows



In [10]:
#Spark SQL based
##in ascending order
crimeCategory_desc = spark.sql('''
                SELECT category, COUNT(*) AS count 
                FROM sf_crime 
                GROUP BY category 
                ORDER BY count ASC''')

#display result
crimeCategory_desc.show()

+--------------------+-----+
|            category|count|
+--------------------+-----+
|                TREA|   14|
|PORNOGRAPHY/OBSCE...|   59|
|            GAMBLING|  348|
|SEX OFFENSES, NON...|  431|
|           EXTORTION|  741|
|             BRIBERY|  813|
|          BAD CHECKS|  925|
|     FAMILY OFFENSES| 1183|
|             SUICIDE| 1292|
|           LOITERING| 2430|
|        EMBEZZLEMENT| 2988|
|               ARSON| 3931|
|         LIQUOR LAWS| 4083|
|             RUNAWAY| 4440|
|          KIDNAPPING| 5346|
|DRIVING UNDER THE...| 5672|
|   RECOVERED VEHICLE| 8716|
|         DRUNKENNESS| 9826|
|  DISORDERLY CONDUCT|10040|
|SEX OFFENSES, FOR...|11742|
+--------------------+-----+
only showing top 20 rows



#### Observation:
Larceny/Theft category was reported the most, whereas TREA has the lowest count in SF area.
In addition, we can see that VEHICLE THEFT ranked 4th in among all types of crimes, it may suggest that it is not safe to rent a car to travel in San Francisco.

### <a id='2.3'>2.3. Number of crime for each Sunday at SF Downtown</a>  

In [11]:
##google search appears SF downtown (financial district)'s coordinates to be: 37.7946° N, 122.3999° W
##therefore, we can assume SF downtown is centered at 37.7946° N, 122.3999° W
##within a range of 1.19 square kilometers (approx. 119ha) is called 'SF downtown' / largest distance to the center point: (1.19/3.14)**0.5 = 0.616 km
##assume 1° ≈ 111 km at longitude & latitude

#obtain longitude & latitude
crimesSFdt_x = spark.sql("SELECT DayOfWeek, Date, Address, X FROM sf_crime")
crimesSFdt_y = spark.sql("SELECT DayOfWeek, Date, Address, Y FROM sf_crime")
#crimesSFdt_x.show()
#crimesSFdt_y.show()

from pyspark.sql.types import FloatType

def x_square_float(x):
    return ((float(x)-(-122.3999))*111)**2 

x_square_udf_float = udf(lambda z: x_square_float(z), FloatType())
crimes3_df = crimesSFdt_x.select('DayOfWeek', 'Date', 'Address', 'X', x_square_udf_float('X').alias('x_float_squared'))
#crimes3_df.show()

#convert to Pandas dataframe
#crimes3_pd_df = crimes3_df.toPandas()
#display(crimes3_pd_df)

def y_square_float(y):
    return ((float(y)-37.7946)*111)**2 

y_square_udf_float = udf(lambda z: y_square_float(z), FloatType())
crimes4_df = crimesSFdt_y.select('DayOfWeek', 'Date', 'Address', 'Y', y_square_udf_float('Y').alias('y_float_squared'))
#crimes4_df.show()

#convert to Pandas dataframe
#crimes4_pd_df = crimes4_df.toPandas()
#display(crimes4_pd_df)

#Create temporary table called df3, df4
crimes3_df.createOrReplaceTempView("df3")
crimes4_df.createOrReplaceTempView("df4")

#write spark.SQL subquery to retrieve distance
crimesSFdt = spark.sql('''SELECT dt.Date, COUNT(distance) AS num_of_crimes 
                       FROM ( 
                         SELECT df3.DayOfWeek, df3.Date, df3.Address, SQRT(df3.x_float_squared + df4.y_float_squared) as distance 
                         FROM df3 
                         INNER JOIN df4 
                         ON df3.address = df4.address 
                         WHERE df3.DayOfWeek = 'Sunday' 
                       ) AS dt 
                       WHERE dt.distance < 0.616 
                       GROUP BY dt.Date 
                       ORDER BY num_of_crimes DESC''')

#display result
crimesSFdt.show()

+----------+-------------+
|      Date|num_of_crimes|
+----------+-------------+
|01/01/2006|        14227|
|04/13/2014|        12163|
|05/25/2014|        10763|
|11/23/2014|         9488|
|11/24/2013|         9212|
|01/16/2011|         9116|
|04/21/2013|         9109|
|10/27/2013|         8890|
|08/03/2014|         8874|
|08/24/2003|         8771|
|10/18/2015|         8710|
|01/25/2015|         8523|
|12/07/2003|         8253|
|09/12/2010|         8176|
|05/15/2016|         8140|
|01/01/2012|         8125|
|07/06/2014|         8097|
|07/26/2009|         8061|
|10/24/2010|         7770|
|09/02/2012|         7627|
+----------+-------------+
only showing top 20 rows



#### Observation:
According to the above results, we can conclude that amongst all Sundays, on 01/01/2006 the number of crimes is at its highest level.
It do make sense since 01/01/2006 appeared to be during New Year Holidays, and public holidays could be the main reason for rising number of crimes.

### <a id='2.4'>2.4. Number of crime in each month</a>
###### Analysis the number of crime in each month of 2015, 2016, 2017.


In [12]:
#Spark SQL based - Soulution for Q4
#counts the number of crimes in each month of 2014, 2015, 2016, 2017

#format the date MM/dd/yyyyy to YYYY-MM-DD 
crimeInYearMonth = spark.sql("SELECT year(to_date(date, 'MM/dd/yyyy')) as year, \
                                     month(to_date(date, 'MM/dd/yyyy')) as month, \
                                     COUNT(*) as num_of_crimes \
                              FROM sf_crime \
                              GROUP BY year, month \
                              HAVING year in (2015, 2016, 2017, 2018) \
                              ORDER BY num_of_crimes DESC")
                             
#display result
crimeInYearMonth.show()

+----+-----+-------------+
|year|month|num_of_crimes|
+----+-----+-------------+
|2015|    3|        13929|
|2015|    8|        13730|
|2015|    5|        13729|
|2017|    3|        13711|
|2015|    1|        13606|
|2016|   10|        13388|
|2015|    7|        13365|
|2017|   10|        13355|
|2015|    6|        13304|
|2017|    5|        13267|
|2017|    7|        13171|
|2015|   10|        13147|
|2017|    1|        13084|
|2016|   12|        13007|
|2016|    1|        12967|
|2015|    4|        12959|
|2017|    4|        12941|
|2015|    9|        12896|
|2017|    8|        12872|
|2016|    5|        12732|
+----+-----+-------------+
only showing top 20 rows



###### Analysis the number of crime in each month FROM 2003-2018

In [13]:
#counts the number of crimes in each month FROM 2003-2018

#format the date MM/dd/yyyyy to YYYY-MM-DD 
crimeInYearMonth = spark.sql("SELECT month(to_date(date, 'MM/dd/yyyy')) as month, \
                                     COUNT(*) as num_of_crimes \
                              FROM sf_crime \
                              GROUP BY month\
                              ORDER BY num_of_crimes DESC")
                             
#display result
crimeInYearMonth.show()

+-----+-------------+
|month|num_of_crimes|
+-----+-------------+
|    1|       197584|
|    3|       197580|
|   10|       190470|
|    4|       189834|
|    5|       187747|
|    8|       187379|
|    9|       183437|
|    7|       181822|
|    2|       178619|
|   11|       175032|
|    6|       174744|
|   12|       170776|
+-----+-------------+



#### Observation:
In year 2015-2018, number of crimes hit its highest level in January and hit its lowest level in December.
I am pretty surprised since January is just next to December. In this case I will suggest travelers to visit SF at end of November since the crime rate in both November and December are low.
In General, we suggest tourists to visit SF in December, June and November.

### <a id='2.5'>2.5. Number of crime on Christmas holiday</a>

Since Christmas (or Feast of the Nativity) is the biggest holiday in a year and thousands of travelers around the world may come to SF during this day. So I analyse the number of crime w.r.t the hour in Christmas day of 2015,2016,2017.

In [14]:
#Spark SQL based - Soulution for Q4
#counts the number of crimes in each month of 2015, 2016, 2017

crimesDate = spark.sql("SELECT res.date, res.hours, COUNT(*) as num_of_crimes \
                        FROM ( \
                               SELECT date, \
                                      CASE WHEN time BETWEEN '00:00' AND '06:00:00' THEN 'early morning' \
                                           WHEN time BETWEEN '06:00' AND '12:00:00' THEN 'morning' \
                                           WHEN time BETWEEN '12:00' AND '18:00:00' THEN 'afternoon' \
                                           WHEN time BETWEEN '18:00' AND '24:00:00' THEN 'night' \
                                      END AS hours \
                               FROM sf_crime \
                               WHERE date IN ('12/25/2015', '12/25/2016', '12/25/2017') \
                        ) AS res \
                        GROUP BY 1,2 \
                        ORDER BY num_of_crimes DESC")

#display result
crimesDate.show()


+----------+-------------+-------------+
|      date|        hours|num_of_crimes|
+----------+-------------+-------------+
|12/25/2015|        night|           88|
|12/25/2017|    afternoon|           84|
|12/25/2015|    afternoon|           83|
|12/25/2017|        night|           75|
|12/25/2016|    afternoon|           70|
|12/25/2017|      morning|           68|
|12/25/2016|        night|           66|
|12/25/2015|early morning|           61|
|12/25/2016|      morning|           57|
|12/25/2015|      morning|           48|
|12/25/2017|early morning|           46|
|12/25/2016|early morning|           35|
+----------+-------------+-------------+



#### Observation:
Analyzed the number of crime with respect to the hours in 12/25/2015, 12/25/2016; 12/25/2017: <br>early morning:between 00:00-6:00 ; <br>morning:between 6:00-12:00; <br> afternoon:between 12:00-18:00; <br>night:between 18:00-00:00;

#### Travel tips to visit SF:
In general, afternoon and night (between 12:00 and 00:00) have higher crime rates, whereas early morning and morning hours appear to be much safer.
My suggestion for travelers who wanna visit San Francisco, avoid arriving at SF city later than 12PM and any night events.
Instead, you should try to arrive SF on the day time and one day earlier and do sightseeing and shopping in the early morning or in the morning.

### <a id='2.6'>2.6. Top-3 dangerous district and the corresponding most frequent types of crimes</a>
Analysis the top-3 dangerous district of SF and analyse the most frequent types of crimes in those district using following steps<br>
(1) Step1: Find out the top-3 danger district  
(2) Step2: Find out the crime event w.r.t category and time (hour) from the result of step 1  
(3) Step3: Give my advice to distribute the police force based on the analysis results.

In [15]:
#(1) Step1: Find out the top-3 danger disrict  
crimePdDistrict_desc = spark.sql('''
    SELECT PdDistrict, COUNT(*) AS Count 
    FROM sf_crime 
    GROUP BY PdDistrict 
    ORDER BY Count DESC ''')

crimePdDistrict_desc.show()

#Top-3 danger district: Southern, Mission, and Norther

+----------+------+
|PdDistrict| Count|
+----------+------+
|  SOUTHERN|399785|
|   MISSION|300076|
|  NORTHERN|272713|
|   CENTRAL|226255|
|   BAYVIEW|221000|
| INGLESIDE|194180|
|TENDERLOIN|191746|
|   TARAVAL|166971|
|      PARK|125479|
|  RICHMOND|116818|
|      null|     1|
+----------+------+



In [16]:
#(2) Step2: find out the crime event w.r.t category and time (hour) from the result of step 1  

##three districts were calculated respectively:
crimes_in_top3 = spark.sql("SELECT res.category, \
                                   res.PdDistrict, \
                                   res.hours, \
                                   COUNT(*) as num_of_crimes \
                            FROM ( \
                                  SELECT category, PdDistrict, \
                                         CASE WHEN time BETWEEN '00:00' AND '06:00:00' THEN 'early morning' \
                                              WHEN time BETWEEN '06:00' AND '12:00:00' THEN 'morning' \
                                              WHEN time BETWEEN '12:00' AND '18:00:00' THEN 'afternoon' \
                                              WHEN time BETWEEN '18:00' AND '24:00:00' THEN 'night' \
                                         END AS hours \
                                  FROM sf_crime \
                                  WHERE PdDistrict IN ('SOUTHERN', 'MISSION', 'NORTHERN') \
                             ) AS res \
                             GROUP BY 1,2,3 \
                             ORDER BY num_of_crimes DESC")

#display result
crimes_in_top3.show()

+--------------+----------+-------------+-------------+
|      category|PdDistrict|        hours|num_of_crimes|
+--------------+----------+-------------+-------------+
| LARCENY/THEFT|  SOUTHERN|    afternoon|        41261|
| LARCENY/THEFT|  SOUTHERN|        night|        39937|
| LARCENY/THEFT|  NORTHERN|        night|        30390|
| LARCENY/THEFT|  NORTHERN|    afternoon|        26616|
| LARCENY/THEFT|  SOUTHERN|      morning|        21001|
| LARCENY/THEFT|   MISSION|        night|        18147|
|  NON-CRIMINAL|  SOUTHERN|    afternoon|        17250|
|OTHER OFFENSES|  SOUTHERN|    afternoon|        17156|
|OTHER OFFENSES|   MISSION|    afternoon|        15244|
| LARCENY/THEFT|   MISSION|    afternoon|        15153|
| LARCENY/THEFT|  NORTHERN|      morning|        14930|
|OTHER OFFENSES|  SOUTHERN|        night|        14646|
|  NON-CRIMINAL|  SOUTHERN|      morning|        13885|
|OTHER OFFENSES|   MISSION|        night|        13095|
|OTHER OFFENSES|  SOUTHERN|      morning|       

Step3 My advice to SF police department: <br> From the above results, police force should be distributed primarily in Southern district, then Nothern district followed by Mission district as those three district are the top-3 district in terms of number of crimes. The police on duty at night should pay attention to LARCENY/THEFT; in the afternoon for OTHER OFFENSES. Residents and tourists in the city should avoid visiting dangerous area from 12:00 PM to midnight.

### <a id='2.7'>2.7. Resolution of Unsolved crime cases</a>
For different category of crime, find the percentage of resolution. Based on the output, give hints to adjust the policy.

In [17]:
#write subqueries tb1 & tb2 to obtain results where resolution is None("Unresolved") or else("resolved")
#left join two tables together
#calculate percentage of resolution
#cnt_un/(cnt_un+cnt_re) is the percentage of unresolved crimes, saved as unsolved_pc
#cnt_re/(cnt_un+cnt_re) is the percentage of resolved crimes, saved as resolved_pc

crimeCategoryPercentage = spark.sql("SELECT tb3.Category, \
                                            cnt_un/(cnt_un+cnt_re) as unsolved_pc, \
                                            cnt_re/(cnt_un+cnt_re) as resolved_pc \
                                     FROM ( \
                                           SELECT tb1.Category, cnt_un, cnt_re \
                                           FROM( \
                                                SELECT Category, COUNT(resolved_or_not) as cnt_un \
                                                FROM (\
                                                      SELECT Category, \
                                                             CASE WHEN Resolution LIKE 'NONE' THEN 'unresolved' \
                                                             ELSE 'resolved' \
                                                             END AS resolved_or_not \
                                                      FROM sf_crime) \
                                                      WHERE resolved_or_not = 'unresolved' \
                                                      GROUP BY Category \
                                                 ) tb1 \
                                           LEFT JOIN \
                                           (SELECT Category, COUNT(resolved_or_not) as cnt_re \
                                            FROM ( \
                                                  SELECT Category, \
                                                         CASE WHEN Resolution LIKE 'NONE' THEN 'unresolved' \
                                                         ELSE'resolved' \
                                                         END AS resolved_or_not \
                                                  FROM sf_crime) \
                                                  WHERE resolved_or_not = 'resolved' \
                                                  GROUP BY Category \
                                           ) tb2 \
                                           ON tb1.Category = tb2.Category \
                                    ) tb3 \
                                    ORDER BY unsolved_pc DESC")
 
#display result
crimeCategoryPercentage.show()


+--------------------+------------------+-------------------+
|            Category|       unsolved_pc|        resolved_pc|
+--------------------+------------------+-------------------+
|   RECOVERED VEHICLE|0.9308168884809546|0.06918311151904544|
|       VEHICLE THEFT| 0.916099271733464|0.08390072826653608|
|       LARCENY/THEFT|0.9114971859597709|0.08850281404022912|
|      SUSPICIOUS OCC|0.8824275272239073|0.11757247277609269|
|           VANDALISM|0.8779155429565997|0.12208445704340035|
|            BURGLARY|0.8373441989010629|0.16265580109893713|
|               ARSON|0.8074281353345205|0.19257186466547951|
|          BAD CHECKS|0.8043243243243243|0.19567567567567568|
|           EXTORTION|0.7786774628879892| 0.2213225371120108|
|        NON-CRIMINAL|0.7756616021114202| 0.2243383978885798|
|               FRAUD|0.7716768571566126|0.22832314284338742|
|             ROBBERY|0.7711529167487068|0.22884708325129324|
|             SUICIDE| 0.760061919504644|0.23993808049535603|
|       

#### Suggestions to adjust the policy:
1) Police force should focus more on cimes like RECOVERED VEHICLE, VEHICLE THEFT, and LARCENY/THEFT since these three categories have the highest 'unresolved' percentages.

2) RECOVERED VEHICLE, VEHICLE THEFT are the Top-2 unresolved crimes, which indicates that crimes related to vehicles require additional attention from the police department.

3) Police should take some actions to raise the awareness of vehicle owners, as a result of common vehicles safty/security issues.

### <a id='2.8'>2.8. My tips for tourists in SF</a>

#### Travel tips to visit SF:
Based on the previous results:
<br>Tourists should avoid visiting SF on Public Holidays. 
<br>Tourists should arrange most of their events and sightseeing in the morning or afternoon.
<br>Tourists should consider booking hotels in PARK district or RICHMOND district and avoid living in SOUTHERN district and MISSION district. 
<br>Tourists should pay attention to their vehicle and try not to leave any valuables in the vehicle since there are many unsolved VEHICLE related crimes in SF.



# <a id='3'>3. Venue Recommandation</a>

### <a id='3.1'>3.1. Hotel Booking Recommadation</a>

In [18]:
##in descending order
crimePdDistrict_desc = spark.sql(
    '''SELECT PdDistrict, COUNT(*) AS Count 
    FROM sf_crime 
    GROUP BY PdDistrict 
    ORDER BY Count DESC''')

#display result
crimePdDistrict_desc.show()

+----------+------+
|PdDistrict| Count|
+----------+------+
|  SOUTHERN|399785|
|   MISSION|300076|
|  NORTHERN|272713|
|   CENTRAL|226255|
|   BAYVIEW|221000|
| INGLESIDE|194180|
|TENDERLOIN|191746|
|   TARAVAL|166971|
|      PARK|125479|
|  RICHMOND|116818|
|      null|     1|
+----------+------+



#### Observation:
Southern Pd District has the highest number of crimes, whereas crimes in Richmond are the lowest.
Thus I will avoid book hotels in Southern district as it is the most dangerous district.
I will suggest book hotels in Richmond district as it is the safest district.

### <a id='3.2'>3.2. Venue Recommadation</a>

#### Following 3.1, Since we decided to book an hotel in Central Richmond, we will explore some venues in Richmond distict using Foursquare API.

##### google search appears the Central Richmond, SF have an latitude = 37.780786 and an longitude = -122.483570

In [19]:
latitude = 37.780786 
longitude = -122.483570

##### add my credentials inorder to use Foursquare API

In [35]:
CLIENT_ID = 'HAHAHAHA' # use your own Foursquare ID
CLIENT_SECRET = 'HAHAHAHA' # use your own Foursquare Secret
VERSION = '20200203'
radius = 500
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: HAHAHAHA
CLIENT_SECRET:HAHAHAHA


##### form the query query request url

In [36]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=HAHAHAHA&client_secret=HAHAHAHA&ll=37.780786,-122.48357&v=20200203&radius=500&limit=30'

In [26]:
import requests

In [27]:
results = requests.get(url).json()
'There are {} around Ecco restaurant.'.format(len(results['response']['groups'][0]['items']))

'There are 30 around Ecco restaurant.'

In [28]:
items = results['response']['groups'][0]['items']
items[0]

{'reasons': {'count': 0,
  'items': [{'summary': 'This spot is popular',
    'type': 'general',
    'reasonName': 'globalInteractionReason'}]},
 'venue': {'id': '5dabf0942ab63a0007650772',
  'name': 'Aziza Restaurant',
  'location': {'address': '5800 Geary Blvd',
   'lat': 37.780335,
   'lng': -122.481575,
   'labeledLatLngs': [{'label': 'display',
     'lat': 37.780335,
     'lng': -122.481575}],
   'distance': 182,
   'postalCode': '94121',
   'cc': 'US',
   'city': 'San Francisco',
   'state': 'CA',
   'country': 'United States',
   'formattedAddress': ['5800 Geary Blvd',
    'San Francisco, CA 94121',
    'United States']},
  'categories': [{'id': '4bf58dd8d48988d1c3941735',
    'name': 'Moroccan Restaurant',
    'pluralName': 'Moroccan Restaurants',
    'shortName': 'Moroccan',
    'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/moroccan_',
     'suffix': '.png'},
    'primary': True}],
  'photos': {'count': 0, 'groups': []}},
 'referralId': 'e-0-5dabf0942ab63a0007

##### Let's retrieve the venue's information nearby

In [29]:
from pandas.io.json import json_normalize

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

dataframe = json_normalize(items) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Aziza Restaurant,Moroccan Restaurant,5800 Geary Blvd,37.780335,-122.481575,"[{'label': 'display', 'lat': 37.780335, 'lng':...",182,94121,US,San Francisco,CA,United States,"[5800 Geary Blvd, San Francisco, CA 94121, Uni...",NaN,NaN,5dabf0942ab63a0007650772
1,Fiorella,Italian Restaurant,2339 Clement St,37.781887,-122.484510,"[{'label': 'display', 'lat': 37.78188735290587...",147,94121,US,San Francisco,CA,United States,"[2339 Clement St, San Francisco, CA 94121, Uni...",NaN,NaN,56995964498e62132be29321
2,PPQ Dungeness Island,Seafood Restaurant,2332 Clement St,37.782104,-122.484461,"[{'label': 'display', 'lat': 37.78210375510667...",166,94121,US,San Francisco,CA,United States,"[2332 Clement St (at 24th Ave), San Francisco,...",at 24th Ave,Outer Richmond,42ec1480f964a520a1261fe3
3,Seakor Polish Deli & European Foods,Deli / Bodega,5957 Geary Blvd,37.780034,-122.483553,"[{'label': 'display', 'lat': 37.78003365266694...",83,94121,US,San Francisco,CA,United States,"[5957 Geary Blvd (at 24th Ave), San Francisco,...",at 24th Ave,Outer Richmond,4af5d67ff964a52094fd21e3
4,Richmond Nail Salon,Cosmetics Shop,5856 Geary Blvd,37.780252,-122.482486,"[{'label': 'display', 'lat': 37.78025182776000...",112,94121,US,San Francisco,CA,United States,"[5856 Geary Blvd (at 23rd Ave), San Francisco,...",at 23rd Ave,NaN,4c9966e3db10b60cfa258a6d
5,Hodala,Taiwanese Restaurant,5801 Geary Blvd,37.780165,-122.481625,"[{'label': 'display', 'lat': 37.78016542223118...",184,94121,US,San Francisco,CA,United States,"[5801 Geary Blvd, San Francisco, CA 94121, Uni...",NaN,NaN,5a94cffea6ec982aa0956a51
6,Dragon Beaux 俏龍軒,Dim Sum Restaurant,5700 Geary Blvd,37.780470,-122.480633,"[{'label': 'display', 'lat': 37.780469941427, ...",260,94121,US,San Francisco,CA,United States,"[5700 Geary Blvd (at 21st Ave), San Francisco,...",at 21st Ave,NaN,54f08677498ecc627d4dfc22
7,Tommy's Mexican Restaurant,Mexican Restaurant,5929 Geary Blvd,37.779999,-122.483160,"[{'label': 'display', 'lat': 37.77999916404445...",94,94121,US,San Francisco,CA,United States,"[5929 Geary Blvd (at 23rd Ave), San Francisco,...",at 23rd Ave,NaN,3fd66200f964a520a3ec1ee3
8,Gordo Taqueria,Burrito Place,2252 Clement St,37.782162,-122.483655,"[{'label': 'display', 'lat': 37.78216178702004...",153,94121,US,San Francisco,CA,United States,"[2252 Clement St (btwn 23rd & 24th Ave), San F...",btwn 23rd & 24th Ave,NaN,4a062117f964a520bb721fe3
9,Pizzetta 211,Pizza Place,211 23rd Ave,37.783694,-122.482879,"[{'label': 'display', 'lat': 37.78369393632517...",329,94121,US,San Francisco,CA,United States,"[211 23rd Ave (at California St), San Francisc...",at California St,NaN,40df5f80f964a52096011fe3


#### Let's visualize these venues on the map around our location

In [30]:
import folium
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the Central Richmond
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Central Richmond',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the retrieved restaurants as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

##### Let's look up the ratings of these shops 

In [31]:
venue_id_list=[results['response']['groups'][0]['items'][i]['venue']['id'] for i in range(LIMIT)]
for i,venue_id in enumerate(venue_id_list):
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

    result = requests.get(url).json()
    try:
        print('%s has a rating of %.1f' % (results['response']['groups'][0]['items'][i]['venue']['name'],result['response']['venue']['rating']))
    except:
        print('This venue has not been rated yet.')

Aziza Restaurant has a rating of 9.1
Fiorella has a rating of 8.9
PPQ Dungeness Island has a rating of 8.6
Seakor Polish Deli & European Foods has a rating of 8.3
Richmond Nail Salon has a rating of 8.2
Hodala has a rating of 8.1
Dragon Beaux 俏龍軒 has a rating of 8.4
Tommy's Mexican Restaurant has a rating of 8.0
Gordo Taqueria has a rating of 8.0
Pizzetta 211 has a rating of 9.2
Royal Ground Coffee House has a rating of 7.8
Daigo Sushi has a rating of 7.9
Pearl has a rating of 8.6
Ton Kiang has a rating of 7.7
Home has a rating of 8.1
Clement Nursery has a rating of 8.5
Blackwell's Wines and Spirits has a rating of 8.3
Hard Knox Cafe has a rating of 7.8
Turtle Tower Restaurant has a rating of 7.7
Red Tavern has a rating of 7.5
Bambu Desserts & Drinks has a rating of 7.4
4 Star Theatre has a rating of 7.4
Moscow & Tbilisi Russian Bakery has a rating of 8.3
La Vie has a rating of 7.4
Violet’s has a rating of 7.4
Lokma has a rating of 8.5
Jang Soo BBQ has a rating of 7.7
Ilana Coffee has 

##### We can see that Pizzetta 211 has the highest rating, let's explore it further.

In [32]:
venue_id = '40df5f80f964a52096011fe3' # ID of Pizzetta 211
url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)

result = requests.get(url).json()
#count number of tips
result['response']['venue']['tips']['count']

92

In [33]:
## Ecco Tips
limit = 30 # set number of tips to retrieve from foursquare service
url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&v={}&limit={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION, limit)

results = requests.get(url).json()

##### Explore some review of Pizzetta 211

In [34]:
pd.set_option('display.max_colwidth', -1)

tips = results['response']['tips']['items']
tips_df = json_normalize(tips) # json normalize tips

# columns to keep
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName', 'user.gender', 'user.id']
tips_filtered = tips_df.loc[:, filtered_columns]

# display tips
tips_filtered

# Since I am using a personal developer account, 
# then I can access only 2 of the restaurant's tips, instead of all 30 tips.

C:\Users\gaoji_000\Anaconda2\envs\Python36\lib\site-packages\pandas\core\indexing.py:1404: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


,text,agreeCount,disagreeCount,id,user.firstName,user.lastName,user.gender,user.id
0,"Delicious personal pizzas, usually they have one or two on the menu with farm egg - yum! The wait for sit down is long, so consider ordering to go!",4,0,584308132e9fde4e367ee38f,Kate,A,NaN,58597848
1,"The white bean, artichoke salad w/ arugula is delish if they have it as a special. Sitting outside is nice on a sunny day. And they have Unti Barbera by the glass which you don't find at many places!",2,0,53261ee011d2596f0052b58b,Marina,C,NaN,2485273


#### Seems pretty good, I will buy piazza from this shop during my stay in SF

# <a id='4'>4. Conclusion</a>

In this notebook we performmed an anakysis of a large-sized data set of SF crimes using Spark and Spark SQL.

<br>We explored and provided some useful findings including SF top-3 common crimes, SF top-3 danger districts, number of crimes around downtown SF, hours (time) for particular dates, and resolved & unresolved percentages for different crime categories, etc.

<br>Based on our results, we suggest that advertisings on safety topics are needed to raise awareness primarily among residents in SOUTHERN, MISSION, and NORTHERN districts.Furthermore, police officers should focus more on cimes with high unsolved percentage like RECOVERED VEHICLE, VEHICLE THEFT, and LARCENY/THEFT. In general, we suggest tourists should avoid visiting the city or walking around the streets at night, because the chances of being involved in a crime are higher than that during other daytime. In addition, tourists should pay extra attention during holidays and VEHICLE related crimes.

<br>Lastly, we explored some of the venues in the most safe district (the RICHMOND district) of SF using Foursuqare API, we extract the ratings and reviews of those venues and we recommand some of the shops with high ratings. Also we visualized those venues on the Map using Folium API.

